# Machine Learning: Text Classification Assignment

In [14]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.linear_model import LogisticRegression
from nltk.corpus.reader.plaintext import CategorizedPlaintextCorpusReader

import requests
from bs4 import BeautifulSoup

In [2]:
def preprocess(docs): 
    lemmatizer = WordNetLemmatizer() 
    stemmer = SnowballStemmer('english') 
    
    preprocessed = []
    for doc in docs: 
        tokenized = word_tokenize(doc)
        cleaned = [stemmer.stem(lemmatizer.lemmatize(token.lower())) for token in tokenized 
               if not token.lower() in stopwords.words('english') 
               if token.isalpha()]

        untokenized = " ".join(cleaned)
        preprocessed.append(untokenized)
        
    return preprocessed

### Use the CategorizedPlaintextCorpusReader to import the AP_News corpus.

In [3]:
path = 'AP_News'
DOC_PATTERN = r'.*\.txt'
CAT_PATTERN = r'([\w_\s]+)/.*'

corpus = CategorizedPlaintextCorpusReader(path, DOC_PATTERN, cat_pattern=CAT_PATTERN)

### Create two separate lists - one containing the text from each document and another containing the category of each article in the corpus.

In [4]:
docs = [corpus.raw(fileid) for fileid in corpus.fileids()]

categories = [corpus.categories(fileid)[0] for fileid in corpus.fileids()]

### Preprocess the corpus, ensuring to include the following steps.

- Word tokenize the documents.
- Lemmatize, stem, and lowercase all tokens.
- Remove punctuation and stop words.

In [5]:
preprocessed = preprocess(docs)

### Split the data into training and testing sets with the size of the test set being 30% of the records.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(preprocessed, categories, test_size=0.30)

### Construct a pipeline that TF-IDF vectorizes the text and trains a Random Forest classification model.

In [8]:
model = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('clf', LogisticRegression()),
])

model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
  

### Generate predictions on the test set and print a classification report to evaluate how well the model performed.

In [9]:
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

      health       0.81      0.59      0.68        22
    politics       0.86      0.75      0.80        16
      sports       1.00      0.76      0.87        17
        tech       0.39      0.82      0.53        11

    accuracy                           0.71        66
   macro avg       0.77      0.73      0.72        66
weighted avg       0.80      0.71      0.73        66



### Perform 10-fold cross validation and obtain the averge F1 score across all the folds.

In [12]:
scores = cross_val_score(model, preprocessed, categories,
                         cv=10, scoring = 'f1_macro')
scores.mean()

0.8187741262005968

### Ingest, preprocess, and predict the topic of the article at the following URL.

In [16]:
url = 'https://www.nytimes.com/2019/11/25/business/uber-london.html'

In [20]:
response = requests.get(url)
content = response.text

tags = ['p', 'li']
soup = BeautifulSoup(content, 'lxml')
text_list = [tag.get_text() for tag in soup.find_all(tags)]
text = ' '.join(text_list)

In [23]:
cleaned = preprocess([text])
model.predict(cleaned)[0]

'tech'